In [1]:
!pip install transformers==4.30

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install datasets sentencepiece accelerate bitsandbytes peft trl mlflow config jsonlines

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

#from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from datasets import Dataset
from datasets import load_dataset
from pprint import pprint
from tqdm import tqdm
logger = logging.getLogger(__name__)
global_config = None

In [34]:
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM
from transformers.trainer_callback import TrainerCallback
from datasets import load_dataset, DatasetDict, Dataset
from transformers import BitsAndBytesConfig
from trl import SFTTrainer
import mlflow

In [12]:
dataset_name = 'qwedsacf/grade-school-math-instructions'
model_name = 'openlm-research/open_llama_3b_v2'


In [13]:
raw_dataset = load_dataset(dataset_name, split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/8792 [00:00<?, ? examples/s]

In [14]:
import pandas as pd

instruction_dataset_df = pd.DataFrame(raw_dataset)
instruction_dataset_dict = instruction_dataset_df.to_dict()

In [15]:
instruction_dataset_df.head(2)

,INSTRUCTION,RESPONSE,SOURCE
0,This math problem has got me stumped: Natalia ...,Natalia sold 48/2 = 24 clips in May.\nNatalia ...,grade-school-math
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $0.2 per minute.\nWorking 5...,grade-school-math


In [23]:
instruction_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8792 entries, 0 to 8791
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   INSTRUCTION  8792 non-null   object
 1   RESPONSE     8792 non-null   object
 2   SOURCE       8792 non-null   object
dtypes: object(3)
memory usage: 206.2+ KB


In [17]:
instruction_dataset_df['INSTRUCTION'].apply(lambda x: len(x.split())).max()

236

In [19]:
def form_finetuning_dataset(dataset_dict: dict, question_key: str, answer_key: str) -> Dataset:
    instruction_template = """{question}"""

    prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:

    {instruction}

    ### Response:\n"""

    num_samples = len(dataset_dict[question_key])
    finetuning_dataset_list = []
    for i in range(num_samples):
        question = dataset_dict[question_key][i]
        instruction = instruction_template.format(question=question)
        prompt = prompt_template.format(instruction=instruction)
        response = dataset_dict[answer_key][i] + "\n### End"
        text = prompt + response
        finetuning_dataset_list.append({"instruction": instruction, "response": response, "text": text})

    finetuning_dataset = Dataset.from_list(finetuning_dataset_list)

    print("One sample in the finetuning dataset:")
    pprint(finetuning_dataset[0])

    return finetuning_dataset

In [20]:
finetuning_dataset = form_finetuning_dataset(instruction_dataset_dict, question_key = "INSTRUCTION", answer_key = "RESPONSE")

One sample in the finetuning dataset:
{'instruction': 'This math problem has got me stumped: Natalia sold clips to '
                '48 of her friends in April, and then she sold half as many '
                'clips in May. How many clips did Natalia sell altogether in '
                'April and May?\n'
                'Can you show me the way?',
 'response': 'Natalia sold 48/2 = 24 clips in May.\n'
             'Natalia sold 48+24 = 72 clips altogether in April and May.\n'
             '### End',
 'text': 'Below is an instruction that describes a task. Write a response that '
         'appropriately completes the request.\n'
         '\n'
         '    ### Instruction:\n'
         '    \n'
         '    This math problem has got me stumped: Natalia sold clips to 48 '
         'of her friends in April, and then she sold half as many clips in '
         'May. How many clips did Natalia sell altogether in April and May?\n'
         'Can you show me the way?\n'
         '    \n'
     

In [35]:
split_dataset = finetuning_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
test_dataset = split_dataset['test'].train_test_split(test_size=0.5, shuffle=True, seed=42)

dataset = DatasetDict({
    'train' : split_dataset['train'],
    'test' : test_dataset['train'],
    'eval' : test_dataset['test'],
})

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'response', 'text'],
        num_rows: 7912
    })
    test: Dataset({
        features: ['instruction', 'response', 'text'],
        num_rows: 440
    })
    eval: Dataset({
        features: ['instruction', 'response', 'text'],
        num_rows: 440
    })
})

In [37]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [38]:
#If only targeting attention blocks of the model
target_modules = ["q_proj", "v_proj"]

#If targeting all linear layers
#target_modules = ['q_proj','k_proj','v_proj','o_proj','gate_proj','down_proj','up_proj','lm_head']

lora_config = LoraConfig(r=16,
                        target_modules = target_modules,
                        lora_alpha=8,
                        lora_dropout=0.05,
                        bias="none",
                        task_type="CAUSAL_LM")

In [39]:
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = 'adamw_hf'
learning_rate = 1e-5
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [41]:
training_args = TrainingArguments(
    output_dir="01-experiment",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    num_train_epochs = 1.0,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [42]:
nf4_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_use_double_quant=True,
  bnb_4bit_compute_dtype=torch.bfloat16
)

In [43]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/512k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/330 [00:00<?, ?B/s]

1

In [44]:
model = LlamaForCausalLM.from_pretrained(
    model_name, device_map='auto', quantization_config=nf4_config,
)

config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.85G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [45]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,324,800 || all params: 3,431,798,400 || trainable%: 0.15516062948219803


In [46]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    dataset_text_field="text",
    max_seq_length=256,
    args=training_args,
    )

Using pad_token, but it is not set yet.


Map:   0%|          | 0/7912 [00:00<?, ? examples/s]

Map:   0%|          | 0/440 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:247: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [47]:
#Upcast layer norms to float 32 for stability
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.float32)

In [48]:
# Initiate the training process
with mlflow.start_run(run_name='02-LoRA-Experiment'):
    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
0,No log,1.317377


In [50]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [51]:
model.push_to_hub("open_llama_3b_v2-Fine-Tuned-Grade_School_Math_Instructions")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'README.md'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_config.json'. This can cause undesired inconsistencies in your repo.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/_commit_api.py:282: UserWarning: About to update multiple times the same file in the same commit: 'adapter_model.bin'. This can cause undesired inconsistencies in your repo.
  warnings.warn(


adapter_model.bin:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/21.3M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Logisx/open_llama_3b_v2-Fine-Tuned-Grade_School_Math_Instructions/commit/0f2a4feb382ed8e8e35c5a97bfc0211d08a8a4b3', commit_message='Upload model', commit_description='', oid='0f2a4feb382ed8e8e35c5a97bfc0211d08a8a4b3', pr_url=None, pr_revision=None, pr_num=None)

In [52]:
evaluation_results = trainer.evaluate()


NameError: ignored

In [53]:
import math
print(f"Perplexity: {math.exp(evaluation_results['eval_loss']):.2f}")

Perplexity: 3.73


In [43]:
#dbutils.fs.ls("01-experiment")

# COMMAND ----------

model_path = 'openlm-research/open_llama_3b_v2'

# COMMAND ----------

tokenizer_2 = LlamaTokenizer.from_pretrained(model_path)
tokenizer_2.add_special_tokens({'pad_token': '[PAD]'})

# COMMAND ----------

model_2 = LlamaForCausalLM.from_pretrained(
    model_path, device_map='auto', quantization_config=nf4_config,
)

# COMMAND ----------

peft_model_id = '/content/01-experiment/checkpoint-129/'

# COMMAND ----------

peft_model = PeftModel.from_pretrained(model_2, peft_model_id)

RuntimeError: ignored

In [ ]:
test_strings = dataset['test'][0:3]['instruction']

In [ ]:
predictions = []
for test in test_strings:
  prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  {}

  ### Response:""".format(test)
  input_ids = tokenizer_2(prompt, return_tensors="pt").input_ids.to('cuda')

  generation_output = model_2.generate(
      input_ids=input_ids, max_new_tokens=156
  )
  predictions.append(tokenizer_2.decode(generation_output[0]))

In [ ]:
def extract_response_text(input_string):
    start_marker = '### Response:'
    end_marker = '###'

    start_index = input_string.find(start_marker)
    if start_index == -1:
        return None

    start_index += len(start_marker)

    end_index = input_string.find(end_marker, start_index)
    if end_index == -1:
        return input_string[start_index:]

    return input_string[start_index:end_index].strip()

In [ ]:

for i in range(3):
  pred = predictions[i]
  text = test_strings[i]
  print(text+'\n')
  print(extract_response_text(pred))
  print('--------')

In [54]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=300):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [61]:
dataset['eval'][0]['instruction']

'An airport has only 2 planes that fly multiple times a day. Each day, the first plane goes to Greece for three-quarters of its flights, and the remaining flights are split equally between flights to France and flights to Germany. The other plane flies exclusively to Poland, and its 44 trips only amount to half the number of trips the first plane makes throughout each day. How many flights to France does the first plane take in one day?'

In [64]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:

    {}

    ### Response:\n"""
test_text = template.format(dataset['eval'][1]['instruction'])
print("Question input (test):", test_text)
print(f"Correct answer: {dataset['eval'][1]['response']}")
print("Model's answer: ")
print(inference(test_text, trainer.model, tokenizer))

Question input (test): Below is an instruction that describes a task. Write a response that appropriately completes the request.

    ### Instruction:
    
    Can you break down the solution for me for this math problem: Craig and his brother take turns spelling out the longest letter words they know and counting the number of the letters in those words. After ten rounds, Craig has spelled out 20 words with 15 letters each. If Craig's brother has spelled words with a total count of letters 50 more than Craig, calculate the total number of letters in the words they've spelled after the ten rounds.
    
    ### Response:

Correct answer: After ten rounds, Craig has spelled out 20*15 = 300 letters from the longest letter words he knows.
Since Craig's brother spelled out words with a total count of 50 more letters than Craig's words, he got 300+50 = 350 letters.
The brothers spelled out words with a total count of 350+300 = 650 letters.
### End
Model's answer: 
Craig and his brother take 

In [40]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def exact_match_ratio(predictions, references):
    total_cases = len(predictions)
    correct_cases = sum(1 for pred, ref in zip(predictions, references) if pred == ref)
    return correct_cases / total_cases if total_cases > 0 else 0.0


def calculate_metric_on_test_ds(dataset, model, tokenizer,
                                batch_size=16,
                                device=device,
                                question_key="question",
                                answer_key="answer"):

    input_batches = list(generate_batch_sized_chunks(dataset[question_key], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[answer_key], batch_size))

    predictions_batches = []
    references_batches = []
    for input_batches, target_batch in tqdm(zip(input_batches, target_batches), total=len(input_batches)):

        inputs = tokenizer(input_batches,
                           max_length=128,
                           truncation=True,
                           padding="max_length",
                           return_tensors="pt")

        regex = model.generate(input_ids=inputs["input_ids"].to(device),
                                 attention_mask=inputs["attention_mask"].to(device),
                                 num_beams=8,
                                 max_length=512)

        decoded_regex = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in regex]

        decoded_regex = [d.replace("", " ") for d in decoded_regex]

        predictions_batches.append(decoded_regex[0])
        references_batches.append(target_batch[0])


    score = exact_match_ratio(predictions_batches, references_batches)
    return score

In [41]:
score = calculate_metric_on_test_ds(dataset['test'][0:16],
                                    model=trainer.model,
                                    tokenizer=tokenizer,
                                    batch_size = 2,
                                    question_key='instruction',
                                    answer_key='response')


print("Exact matches persentage: ", score)

 12%|█▎        | 1/8 [02:38<18:31, 158.76s/it]


RuntimeError: ignored

In [42]:
dataset['test'][0]

{'instruction': '### Generate a regex for this description:\n    IPv6 text representation of addresses without compression from RFC 1884. This regular expression doesn\'t allow IPv6 compression (&quot;::&quot;) or mixed IPv4 addresses.\nMatch examples:\n- "FEDC:BA98:7654:3210:FEDC:BA98:7654:3210"\n- "1080:0:0:0:8:800:200C:417A"\n- "0:0:0:0:0:0:0:1"\nNon-match examples:\n- "128.0.0.1"\n- "FEDC:BA98:7654:3210:FEDC:BA98:7654:3210:AA31"',
 'response': '^([0-9A-Fa-f]{1,4}:){7}[0-9A-Fa-f]{1,4}$\n### End',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n    ### Instruction:\n    \n    ### Generate a regex for this description:\n    IPv6 text representation of addresses without compression from RFC 1884. This regular expression doesn\'t allow IPv6 compression (&quot;::&quot;) or mixed IPv4 addresses.\nMatch examples:\n- "FEDC:BA98:7654:3210:FEDC:BA98:7654:3210"\n- "1080:0:0:0:8:800:200C:417A"\n- "0:0:0:0:0:0:0:1"\nNon-match